# RSI_Hedge_v2.7

## Setup

### Packages

In [1]:
from utils.packages import *

### Inputs and Parameters

#### Read Yaml files

In [2]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)     

## Functions

In [3]:
#--------------------------------------------------------------------------------------------------------------------------
# Get timestamp of the price and segregate it
def get_date_time(resp):
    global data
    time_stamp = resp['time']
    data['ts_date_val'], full_time = time_stamp.split(sep = 'T')
    data['ts_time_val'], _ = full_time.split(sep = '.')

    data['ts_date_val'] = datetime.strptime(data['ts_date_val'], '%Y-%m-%d')
    data['ts_time_val'] = datetime.strptime(data['ts_time_val'], '%H:%M:%S')

    tot_ts = datetime.combine(datetime.date(data['ts_date_val']), datetime.time(data['ts_time_val']))
    tot_ts = tot_ts + timedelta(hours=8)
    data['tot_ts'] = tot_ts.strftime("%Y-%b-%d, %I:%M:%S (%p)")
      
    t2 = datetime.now()    
    data['time_diff'] = (t2 - tot_ts).total_seconds()
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get bid and ask prices
def get_prices(resp):    
    global data
    data['price_bid'] = float(resp['bids'][0]['price'])    
    data['price_ask'] = float(resp['asks'][0]['price'])
    data['price_spread'] = data['price_ask'] - data['price_bid']
    data['price_tick'] = (data['price_ask'] + data['price_bid']) / 2
    return()
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def before_avg_len():
    global data
    data['list_tick_avg'].append(data['price_tick'])
    data['list_spread'].append(data['price_spread'])   
    return()
#--------------------------------------------------------------------------------------------------------------------------

def after_avg_len():
    global data
    data['tick_avg'] = np.mean(data['list_tick_avg'])
    data['tick_sd'] = np.std(data['list_tick_avg'])

    data['spread_avg'] = np.mean(data['list_spread'])
    
    data['list_tick_avg'] = collections.deque([])
    data['list_spread'] = collections.deque([])
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def before_rsi_len():
    global data
    data['list_tick'].append(data['tick_avg'])

    if len(data['list_tick']) == 1:
        data['list_up'].append(0)
        data['list_down'].append(0)
        data['list_AvgGain'].append(0)
        data['list_Avgloss'].append(0)
        data['list_RS'].append(0)
        data['list_RSI'].append(0)
    elif len(data['list_tick']) > 1:        
        old_price = data['list_tick'][len(data['list_tick'])-2]
        new_price = data['tick_avg']
        data['diff'] = new_price - old_price
        
        if data['diff'] > 0:
            data['list_up'].append(new_price - old_price)
            data['list_down'].append(0)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi'])            
        elif data['diff'] < 0:
            data['list_up'].append(0)
            data['list_down'].append(old_price - new_price)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi'])            
        elif data['diff'] == 0:
            data['list_up'].append(0)
            data['list_down'].append(0)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi'])    
            
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def after_rsi_len():
    global data
    data['list_up'].popleft()
    data['list_down'].popleft()
    data['list_AvgGain'].popleft()
    data['list_Avgloss'].popleft()
    data['list_RS'].popleft()
    data['list_RSI'].popleft()
    data['list_tick'].popleft()
    data['list_tick'].append(data['tick_avg'])

    old_price = data['list_tick'][len(data['list_tick'])-2]
    new_price = data['tick_avg']
    data['diff'] = new_price - old_price
    
    if data['diff'] > 0:
        data['list_up'].append(new_price - old_price)
        data['list_down'].append(0)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi'])
    elif data['diff'] < 0:
        data['list_up'].append(0)
        data['list_down'].append(old_price - new_price)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi'])
    elif data['diff'] == 0:
        data['list_up'].append(0)
        data['list_down'].append(0)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi'])            
    
    data['avg_gain'] = data['list_AvgGain'][-1]
    data['avg_loss'] = data['list_Avgloss'][-1]
    data['rs'] = data['list_RS'][-1]
        
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def before_sma():
    global data    
    data['ssma_list'].append(data['tick_avg'])    
    return()
#--------------------------------------------------------------------------------------------------------------------------
def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(data['tick_avg'])

    data['ssma'] = np.mean(data['ssma_list'])
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['ssma_ready']) < 2:
        data['ssma_ready'].append(data['ssma'])
        data['sema_ready'].append(data['sema'])

    elif len(data['ssma_ready']) > 1:
        data['ssma_ready'].popleft()
        data['sema_ready'].popleft()
        data['ssma_ready'].append(data['ssma'])
        data['sema_ready'].append(data['sema'])

        data['ssma_diff'] = data['ssma_ready'][-1] - data['ssma_ready'][len(data['ssma_ready'])-2]
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
        
        data['max_tick'] = np.max(data['ssma_list'])
        data['min_tick'] = np.min(data['ssma_list'])
        
        data['max_gap'] = data['max_tick'] -  data['tick_avg']
        data['min_gap'] = data['min_tick'] - data['tick_avg']
    
    return()
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def before_lma():
    global data
    data['lsma_list'].append(data['tick_avg'])
    return()
#--------------------------------------------------------------------------------------------------------------------------
def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(data['tick_avg'])

    data['lsma'] = np.mean(data['lsma_list'])
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lsma_ready']) < 2:
        data['lsma_ready'].append(data['lsma'])
        data['lema_ready'].append(data['lema'])

    elif len(data['lsma_ready']) > 1:
        data['lsma_ready'].popleft()
        data['lema_ready'].popleft()
        data['lsma_ready'].append(data['lsma'])
        data['lema_ready'].append(data['lema'])

        data['lsma_diff'] = data['lsma_ready'][-1] - data['lsma_ready'][len(data['lsma_ready'])-2]
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
        
        data['ema_diff'] = data['sema'] - data['lema']
        data['sma_diff'] = data['ssma'] - data['lsma']
        data['lma_ready_flag'] = True
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def reset_data():
    global data

    data['num_of_ticks'] = 2
    data['rsi_len'] = 3   
    data['sma_len'] = 3
    data['lma_len'] = 6

    data['rsi_ob'] = 70
    data['rsi_os'] = 30     
    
    #Declaration ------------------------------------------

    data['ts_date_val'] = 0
    data['ts_time_val'] = 0
    data['tot_ts'] = 0
    data['time_diff'] = 0
      
    data['price_ask'] = 0
    data['price_bid'] = 0
    data['price_stop'] = 0
    data['price_tick'] = 0
    data['price_spread'] = 0
    
    data['tick_avg'] = 0
    data['spread_avg'] = 0
    data['tick_sd'] = 0
    
    return()
#==========================================================================================================================

#### API Setup

In [4]:
data = {}
data_df = pd.DataFrame()

data['instrument'] = "EUR_USD"
data['lma_ready_flag'] = False

data['list_tick_avg'] = collections.deque([])
data['list_spread'] = collections.deque([])

data['list_tick'] = collections.deque([])
data['list_up'] = collections.deque([])
data['list_down'] = collections.deque([])
data['list_AvgGain'] = collections.deque([])
data['list_Avgloss'] = collections.deque([])
data['list_RS'] = collections.deque([])
data['list_RSI'] = collections.deque([])

data['ssma_list'] = collections.deque([])
data['ssma_ready'] = collections.deque([])
data['sema_ready'] = collections.deque([])

data['lsma_list'] = collections.deque([])
data['lsma_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])

select_keys = ['tick_avg', 'spread_avg', 'tick_sd', 'diff', 'avg_gain','avg_loss', 'rs', 'rsi', 'sema',  'sema_diff', 'ssma', 'ssma_diff', 'lema', 'lema_diff', 'lsma', 'lsma_diff', 'ema_diff', 'sma_diff', 'max_tick', 'min_tick', 'max_gap', 'min_gap']

access_token = config['oanda_demo_hedge']['token']
accountID = config['oanda_demo_hedge']['account_id']
params = {'instruments': data['instrument']}

api = API(access_token = access_token)
request_stream_data = pricing.PricingStream(accountID=accountID, params=params)
response_stream = api.request(request_stream_data)

## Core Engine

In [5]:
reset_data()

for i, resp in enumerate(response_stream): 
    print(i)
    if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        continue

    # Date and Data ---------------------------------------------
    get_date_time(resp)
    get_prices(resp)
    
    # Build avg tick ---------------------------------------------    
    if len(data['list_tick_avg']) < data['num_of_ticks']:
        before_avg_len()
        continue
    elif len(data['list_tick_avg']) == data['num_of_ticks']:
        after_avg_len()

    # =================================================================
    # Build RSI ---------------------------------------------
    if len(data['list_tick']) < data['rsi_len']:
        before_rsi_len()          
    elif len(data['list_tick']) == data['rsi_len']:
        after_rsi_len() 

    # Build SMA ---------------------------------------------
    if len(data['ssma_list']) < data['sma_len']:
        before_sma()          
    elif len(data['ssma_list']) == data['sma_len']:
        after_sma()  
        
    # Build LMA ---------------------------------------------
    if len(data['lsma_list']) < data['lma_len']:
        before_lma()          
        continue
    elif len(data['lsma_list']) == data['lma_len']:
        after_lma()  
    # =================================================================        
    
    
    if data['lma_ready_flag']:
        new_data = {your_key: data[your_key] for your_key in select_keys}
        live_df = pd.DataFrame([new_data])
        print(live_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
   tick_avg  spread_avg  tick_sd      diff  avg_gain  avg_loss        rs  \
0   1.22565     0.00008      0.0 -0.000033  0.000002  0.000014  0.117647   

         rsi      sema  sema_diff  ...      lema  lema_diff      lsma  \
0  10.526316  1.225663  -0.000019  ...  1.225683  -0.000019  1.225702   

   lsma_diff  ema_diff  sma_diff  max_tick  min_tick   max_gap  min_gap  
0  -0.000016 -0.000019 -0.000032  1.225683   1.22565  0.000033      0.0  

[1 rows x 22 columns]
32
33
34
35
   tick_avg  spread_avg   tick_sd      diff  avg_gain  avg_loss   rs  \
0  1.225633    0.000065  0.000013 -0.000017  0.000002  0.000017  0.1   

        rsi      sema  sema_diff  ...      lema  lema_diff      lsma  \
0  9.090909  1.225645  -0.000019  ...  1.225663  -0.000019  1.225682   

   lsma_diff  ema_diff  sma_diff  max_tick  min_tick  max_gap  min_gap  
0  -0.000019 -0.000019 -0.000027  1.225683  1.225633  0.00005      0

KeyboardInterrupt: 